# Object Detection API Demo
TensorFlow object detector pre-trained on the COCO dataset. Architecture: Inception SSD

Author: ottermegazord
Environment: Google Colab

# Setup

### Install dependencies and packages

In [0]:
!pip install -U --pre tensorflow=="2.*"
!pip install pycocotools

Download `tensorflow/models` into path or `cd` to parent directory of the repository if exists.

In [0]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Compile protobufs

In [0]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

Install the object_detection package

In [0]:
%%bash 
cd models/research
pip install .

### Imports

In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

Import the object detection modules.

In [0]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

Patches:

In [0]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

# Model preparation 